<a href="https://colab.research.google.com/github/vinclaveglia/GraphMBSFinder/blob/master/GraphMBSFinder_19dic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cloning Repository

In [7]:
! git clone https://github.com/vinclaveglia/GraphMBSFinder.git
!pip install torch-geometric
import sys, os, pathlib
sys.path.insert(0, '/content/GraphMBSFinder')
os.listdir()

fatal: destination path 'GraphMBSFinder' already exists and is not an empty directory.


['.config', 'GraphMBSFinder', '12ca.pdb', '4zxn.pdb', 'sample_data']

### Loading libraries and graph neural network

In [8]:
%load_ext autoreload
%autoreload 2

import sys, pathlib
#sys.path.insert(0, '..')
#from base.wp_model import ProteinSegmenter2
#from base.pdb_loader import load_pdb
from base import config
import torch_geometric
import numpy as np
import torch
from base.config import Atom
import pandas as pd
from base.PDB_MBS_preprocessing import get_dataset
from base.input import get_random_train_test_proteins
import warnings
#import biometall
from GCN_and_biometall.gnn_biometall_workflow import *

warnings.filterwarnings('ignore')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

net_CA_path = 'GraphMBSFinder/metadata/model_0.01_50_2_5_0.pth'
net_CB_path = 'GraphMBSFinder/metadata/model_0.01_50_2_5_0.CB.pth_FP0'

net_CA = load_trained_gnn(net_CA_path)
net_CB = load_trained_gnn(net_CB_path)

net_CA.to(device)
net_CB.to(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ProteinSegmenter2(
  (conv1): SAGEConv(22, 50, aggr=mean)
  (conv2): SAGEConv(50, 50, aggr=mean)
  (conv3): SAGEConv(50, 50, aggr=mean)
  (conv4): SAGEConv(50, 3, aggr=mean)
)

### Input:

In [ ]:
from google.colab import files
uploaded = files.upload()
print(uploaded.keys())
t_pdb=list(uploaded.keys())[0]

### Run prediction

In [5]:
path_to_structures = pathlib.Path('.')
path_to_sites = pathlib.Path('.')

db_sites = []

input_graph_CA = get_dataset([t_pdb],
                             atom=Atom.CA,
                             sites=db_sites,
                             path_to_structures=path_to_structures,
                             path_to_sites=path_to_sites)

input_graph_CB = get_dataset([t_pdb],
                             atom=Atom.CB,
                             sites=db_sites,
                             path_to_structures=path_to_structures,
                             path_to_sites=path_to_sites)

input_graph_CA = input_graph_CA[t_pdb]
input_graph_CB = input_graph_CB[t_pdb]

input_graph_CA.to(device)
input_graph_CB.to(device)

pred = gnn_bmtl_predict(t_pdb, input_graph_CA, input_graph_CB, net_CA, net_CB, path_to_structures)

include_and_overlap(pred)

site_lenghts = sorted([int(x) for x in list(pred.keys())], reverse=True)
RES = []
for l in site_lenghts:
    for d in pred[l]:
        if 'included' not in d.keys():
            if 'overlap' not in d.keys():
                print("()",d)
                RES.append(d)
print(RES)

LEN:  369
NON ISOLATI ['ASP_334_A', 'HIS_336_A', 'HIS_339_A', 'HIS_340_A']
UNION ['ASP_334_A', 'HIS_336_A', 'HIS_339_A', 'HIS_340_A', 'CYS_113_A', 'CYS_162_A', 'HIS_336_A', 'ASP_337_A', 'HIS_339_A', 'HIS_340_A', 'HIS_381_A', 'CYS_382_A']
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
BIOMETALL
() {'residues': ['HIS_336_A', 'ASP_337_A', 'HIS_339_A', 'HIS_340_A'], 'n_probes': 7, 'center': array([-26.40843 , -18.37243 ,   5.362428], dtype=float32)}
[{'residues': ['HIS_336_A', 'ASP_337_A', 'HIS_339_A', 'HIS_340_A'], 'n_probes': 7, 'center': array([-26.40843 , -18.37243 ,   5.362428], dtype=float32)}]
